# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп».

В ходе проекта мы должны разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('insurance.csv')

In [3]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Исходя из `describe()` аномальных значений не наблюдается.

In [6]:
data['Возраст'] = data['Возраст'].astype('int64')

Данные полные, всё замечательно, за исключением типа данных в столбце "возраст". Мы сменили тип с `float64` на `int64`, так как возраст чаще используют в виде целых чисел.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Доказательство:**
Пусть $X_1 = XP$, где $X$ - исходная матрица признаков, а $P$ - обратимая матрица, на которую перемножаются признаки.

Начнём с выражения:

$w_1 = ((X_1)^T X_1)^{-1} (X_1)^T y$

Подставим $XP$ вместо $X_1$:

$w_1 = ((XP)^T (XP))^{-1} (XP)^T y$

Используем формулу для транспонирования произведения матриц:

$w_1 = (P^T X^T (XP))^{-1} P^T X^T y$

После воспользуемся сочетательным свойством и вынесем $P^{-1}$ за скобки, после чего воспользуемся свойством ассоциативности и внесём $X^T$ $X$ под одну скобку:

$w_1 = P^{-1}(P^T (X^T X))^{-1} P^T X^T y$

Вынесем $P^T$ за скобки, после чего воспользуемся свойством $A^{-1}A = E$:

$w_1 = P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y$

$= P^{-1}(X^T X)^{-1}X^T y$

Теперь выразим $a_1$:

$a_1 = XP w_1$

$a_1 = XP P^{-1}(X^T X)^{-1}X^T y = X(X^T X)^{-1}X^T y = Xw = a$

Таким образом, мы доказали, что $a_1 = a$.

В итоге, мы можем сделать вывод, что умножение признаков на обратимую матрицу не изменит качество линейной регрессии, так как задача обучения и предсказания остаются неизменными.

In [7]:
X = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']

Достаём фичи и трейны

In [8]:
reg = LinearRegression().fit(X, y)
y_pred = reg.predict(X)
mse_orig = mean_squared_error(y, y_pred)

Обучаем линейную регрессию на изначальных данных, после чего выполняем предикт и вычисляем MSE (также на изначальных данных).

In [9]:
P = np.array([[2, 0, 0, 0], [0, 2, 0, 0], [0, 0, 2, 0], [0, 0, 0, 2]])
X_converted = X @ P

Конвертируем фичи, умножая на обратимую матрицу

In [10]:
reg_converted = LinearRegression().fit(X_converted, y)
y_pred_converted = reg_converted.predict(X_converted)
mse_converted = mean_squared_error(y, y_pred_converted)

Обучаем линейную регрессию на преобразованных данных, после чего выполняем предикт и вычисляем MSE (также на преобразованных данных).

In [11]:
print(f"MSE изначальных фичей: {mse_orig}")
print(f"MSE конвертированных фичей: {mse_converted}")

MSE изначальных фичей: 0.12334688941710859
MSE конвертированных фичей: 0.12334688941710859


In [12]:
if mse_orig == mse_converted:
    print('MSE равны')
else:
    print('MSE не равны')

MSE равны


Получается, что MSE - равны. Доказательство оказалось верным.

## Алгоритм преобразования

**Алгоритм**

Одним из распространенных способов защиты данных клиентов является использование техники, называемой обфускацией (или маскировкой) данных. Целью обфускации данных является преобразование исходных данных в их новое представление, которое затрудняет восстановление исходной информации, сохраняя при этом качество моделей машинного обучения.

Одним из возможных алгоритмов обфускации данных является использование случайного вращения (random rotation). Идея заключается в том, чтобы развернуть исходные данные в n-размерном пространстве с помощью матрицы случайного вращения, так что точки исходных данных оказываются разбросанными в пространстве, и исходную информацию становится трудно восстановить.     

**Обоснование**

Чтобы обосновать, почему качество линейной регрессии не изменится после преобразования данных, мы можем использовать свойство линейной регрессии - ее инвариантность к линейным преобразованиям признаков. Это означает, что если мы умножим признаки на невырожденную матрицу, параметры модели линейной регрессии изменятся, но качество предсказания останется прежним.

Чтобы проверить это, мы можем обучить линейную регрессионную модель на исходных данных и на защищенных данных и сравнить их среднеквадратичную ошибку (MSE) на валидационной выборке. Если MSE для обеих моделей одинаковое или близкое, можно сделать вывод, что качество линейной регрессии не изменилось.

## Проверка алгоритма

In [13]:
def random_rotation_matrix(n_features):
    while True:
        A = np.random.rand(n_features, n_features)
        Q, R = np.linalg.qr(A)
        det = np.linalg.det(Q)
        if det == 0:
            continue
        if det < 0:
            Q[:, 0] *= -1
        break
    return Q

Функция `linalg.qr()` выполняет QR-разложение матрицы.

Данное разложение позволяет представить матрицу A в виде произведения QR, где Q - это ортогональная (унитарная) матрица, а R - это верхнетреугольная матрица. Нам нужна как раз ортогональная матрица.     

Не стоит забывать, что матрица обратима тогда и только тогда, когда она невырождена, то есть её определитель не равен нулю. Значит, если мы получим матрицу с определителем, равным 0, мы просто создадим новую матрицу. А если же определитель будет отрицательным, то умножим первый столбец матрицы на -1, вследствие чего знак определителя изменится.

In [14]:
rrm = random_rotation_matrix(X.shape[1])
X_protected = X @ rrm

Получаем защищенную матрицу

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=222)

In [16]:
X_protected_train, X_protected_val, y_train_prot, y_val_prot = train_test_split(X_protected, y, test_size=0.4, random_state=222)

Делим на валидационные и обучающие выборки

In [17]:
model_original = LinearRegression().fit(X_train, y_train)
y_pred_original = model_original.predict(X_val)
r2_original = r2_score(y_val, y_pred_original)

model_protected = LinearRegression().fit(X_protected_train, y_train)
y_pred_protected = model_protected.predict(X_protected_val)
r2_protected = r2_score(y_val, y_pred_protected)

print("R2 модели с изначальными данными:", r2_original)
print("R2 модели с защищенными данными:", r2_protected)

R2 модели с изначальными данными: 0.39814449127172713
R2 модели с защищенными данными: 0.39814449127182083


Обучаем модель на изначальных данных, после чего на защищенных и сравниваем R2 обеих моделей.

Мы показали, что преобразование исходных признаков линейной регрессионной модели необязательно приводит к изменению качества модели, измеряемого при помощи MSE или R2. Это связано с тем, что умножение признаков на невырожденую матрицу не изменит линейную связь между признаками и целевым признаком. Поэтому можно использовать эту технику преобразования для защиты конфиденциальных данных клиентов, сохраняя при этом качество моделей машинного обучения.